$B_0 = I_n$, since we need it to be PSD and invertible and symmetric.

According to Hessian Update Rule: $B^{k+1} = (I-\mu^k \mathbf{s}^k (\mathbf{y}^k)^\top) B^{k} (I-\mu^k \mathbf{y}^k (\mathbf{s}^k)^\top) + \mu^k \mathbf{s}^k (\mathbf{s}^k)^\top$

 where $\mu^k = \frac{1}{(\mathbf{y}^k)^\top \mathbf{s}^k}$

 where $\mathbf{y}^k = (\nabla q(\mathbf{x}^{k+1})- \nabla {q} (\mathbf{x}^k))$ and $\mathbf{s}^k = (\mathbf{x}^{k+1}-\mathbf{x}^{k})$.

1. What is the minimizer and minimum function value of f(x) and g(x) ? Are both the function convex ? What
is a suitable initial choice of B (denoted by B0, i.e. Replacement of first ?? in the Algorithm 3)? Justify
with proper reasons.


The minimizer for $f(x)$ is $[1,1,...,1]$, achieving the global minimum of 0. This function is convex, being a sum of convex functions. Similarly, for g(x), the minimizer and minimum are also  $[1,1,...,1]$ and 0, respectively. To simplify computations,
$B_0$
  can be chosen directly as the identity matrix, satisfying the requirements of being symmetric, positive semi-definite, and invertible. This allows for an easily computable inverse while ensuring the optimization process remains efficient and effective.

2. Implement Algorithm 3 for solving minx∈Rn f(x), Use backtracking line search with α0 = 0.9, ρ = 0.5, γ = 0.5.
Take the starting point to be x0 = (0, 0, ..., 0). Take n ∈ {1000, 2500, 5000, 7500, 10000}, find minimizer of the
objective function in each case and compute the time taken by the BFGS method with backtracking line search.
Tabulate the time taken by BFGS method for each n.

$\nabla_{x_i}{f(x)} = -8(x_{i-1}^2 - x_i) + 8(x_i^2-x_{i+1})(2x_i) + 2(x_i-1)where i = 2,...,n-1$


for i = n:
$\nabla_{x_n}f(x) = -8(x_{n-1}^2 - x_n)$

for i = 1:
$\nabla_{x_1}f(x) = 8(x_{1}^2 - x_2)(2x_1) + 2(x_1-1)$

In [ ]:
import numpy as np
from numpy import ma
from math import sqrt, pi
from numpy import exp, cos, sin
from numpy.linalg import norm
def fx(xk):
  length = len(xk)
  sum = 0
  for i in range(length-1):
    sum+= 4*(xk[i]**2 - xk[i+1])**2 + (xk[i]-1)**2
  return sum

def gradient_fx(xk):
  n = len(xk)
  grad = []
  grad.append( 16*xk[0]*(xk[0]**2 - xk[1]) + 2*(xk[0]-1) )
  for i in range(1, n-1):
    grad.append(  -8*(xk[i-1]**2 - xk[i]) + 16*xk[i]*(xk[i]**2 - xk[i+1]) + 2*(xk[i]-1) )
  grad.append(-8*(xk[n-2]**2 - xk[n-1]))
  return np.array(grad)

In [ ]:
import time
def get_alpha_bfgs(xk, alpha0, rho, gamma, Bk):
  alpha = alpha0
  pk = -gradient_fx(xk)
  while fx(xk + alpha*Bk@pk) > (fx(xk) + gamma*alpha*gradient_fx(xk)@Bk@pk):
    alpha = rho*alpha
  return alpha


def bfgs(x0, tau, alpha0, rho, gamma, max_iter=5000):
  start_time = time.time()
  xk = np.copy(x0)
  n = len(x0)
  Bk = np.eye(n)
  count = 0
  pk = gradient_fx(xk)
  xks = []
  xks.append(xk)
  while (norm(pk)>tau):
    if count > max_iter:
      break

    alpha = get_alpha_bfgs(xk, alpha0, rho, gamma, Bk)
    xnext = xk - alpha*(Bk@pk)

    sk = xnext - xk
    yk = gradient_fx(xnext) - gradient_fx(xk)
    # BFGS
    Bk = np.dot((np.eye(len(xk)) - np.outer(sk, yk) / np.dot(yk, sk)), np.dot(Bk, (np.eye(len(xk)) - np.outer(yk, sk) / np.dot(yk, sk)))) + np.outer(sk, sk) / np.dot(yk, sk)

    xk = xnext
    pk = gradient_fx(xk)
    xks.append(xk)
    count += 1

  end_time = time.time()
  time_elapsed = end_time - start_time
  return count, xk, fx(xk), xks, time_elapsed

In [ ]:
alpha0 = 0.9
rho = 0.5
gamma = 0.5
ns = [1000, 2500, 5000, 7500, 10000]
tau = 5
for n in ns:
  x0 = [0 for i in range(n)]
  count, minimizer, minimum, xks, time_elapsed=bfgs(x0,tau, alpha0, rho, gamma)
  print("for n :" ,n)
  print("total iterations",count)
  print("Minimizer is",minimizer[0:5],"Minimum value is ",minimum,"Time taken is",time_elapsed)

for n : 1000
total iterations 8
Minimizer is [0.99256635 0.99780927 0.99604053 0.99229373 0.99694739] Minimum value is  1.0026349346100196 Time taken is 1.6736514568328857
for n : 2500
total iterations 13
Minimizer is [0.98938933 0.99273812 0.99454444 0.99306559 0.9953912 ] Minimum value is  0.5988244567273902 Time taken is 33.004910707473755
for n : 5000
total iterations 20
Minimizer is [1.01618197 1.00587515 1.00166145 1.00753486 1.00752898] Minimum value is  1.6667635833283874 Time taken is 329.22562980651855
for n : 7500
total iterations 20
Minimizer is [1.01641243 1.00477564 1.00057778 1.0049301  1.00161134] Minimum value is  1.5508796719251254 Time taken is 1058.6996233463287
for n : 10000
total iterations 22
Minimizer is [1.0060999  0.99241788 0.99342804 0.99354844 1.00005876] Minimum value is  1.3241619094378305 Time taken is 2649.0961079597473


3. Implement Algorithm 3 for solving minx∈Rn g(x), Use backtracking line search with α0 = 0.9, ρ = 0.5, γ = 0.5.
Take the starting point to be x0 = (0, 0, ..., 0). Take n ∈ {1000, 2500, 5000, 7500, 10000}, find minimizer of the
objective function in each case and compute the time taken by the BFGS method with backtracking line search.
Tabulate the time taken by BFGS method for each n.

$\nabla_{x_i}{g(x)} = 2(x_{i} - x_{i}^2)(1-2x_i) + 2(x_{i} - 1)$

when $i = 1,...,n$

In [1]:
import numpy as np
from numpy import ma
from math import sqrt, pi
from numpy import exp, cos, sin
from numpy.linalg import norm
def gx(xk):
  length = len(xk)
  sum = 0
  for i in range(length):
    sum+= (xk[0] - xk[i]**2)**2 + (xk[i]-1)**2
  return sum

def gradient_gx(xk):
  n = len(xk)
  grad = []
  first_grad = 0
  for i in range(1,n):
    first_grad += 2*(xk[0] - xk[i]**2)
  first_grad += 2*(xk[0]-1) + 2*(xk[0] - xk[0]**2)*(1-2*xk[0])
  grad.append( first_grad )
  for i in range(1, n):
    grad.append( -4*(xk[0] - xk[i]**2)*xk[i] + 2*(xk[i]-1) )
  return np.array(grad)


In [6]:
import time
def get_alpha_bfgs(xk, alpha0, rho, gamma, Bk):
  alpha = alpha0
  pk = -gradient_gx(xk)
  while gx(xk + alpha*Bk@pk) > (gx(xk) + gamma*alpha*gradient_gx(xk)@Bk@pk):
    alpha = rho*alpha
  return alpha


def bfgs(x0, tau, alpha0, rho, gamma, max_iter=500):
  start_time = time.time()
  xk = np.copy(x0)
  n = len(x0)
  Bk = np.eye(n)
  count = 0
  pk = gradient_gx(xk)
  xks = []
  xks.append(xk)
  while (norm(pk)>tau):
    if count > max_iter:
      break

    alpha = get_alpha_bfgs(xk, alpha0, rho, gamma, Bk)
    xnext = xk - alpha*(Bk@pk)
    sk = xnext - xk
    yk = gradient_gx(xnext) - gradient_gx(xk)
    # BFGS
    Bk = np.dot((np.eye(len(xk)) - np.outer(sk, yk) / np.dot(yk, sk)), np.dot(Bk, (np.eye(len(xk)) - np.outer(yk, sk) / np.dot(yk, sk)))) + np.outer(sk, sk) / np.dot(yk, sk)

    xk = xnext
    pk = gradient_gx(xk)
    xks.append(xk)
    count += 1

  end_time = time.time()
  time_elapsed = end_time - start_time
  return count, xk, gx(xk), xks, time_elapsed

In [10]:
alpha0 = 0.9
rho = 0.5
gamma = 0.5
ns = [1000, 2500, 5000, 7500, 10000]
tau = 1
elapsedtimelistforgx_inbfgs = []
for n in ns:
  x0 = [0 for i in range(n)]
  count, minimizer, minimum, xks, time_elapsed=bfgs(x0,tau, alpha0, rho, gamma)
  print("When n is",n)
  print("total iterations",count)
  print( "minimizer is ",minimizer[0:5], "minimum value is", minimum ,"time taken is ",time_elapsed)
  elapsedtimelistforgx_inbfgs.append(time_elapsed)

When n is 1000
total iterations 5
minimizer is  [0.99921415 0.99977464 0.99977464 0.99977464 0.99977464] minimum value is 0.00016420670144070933 time taken is  1.0338366031646729
When n is 2500
total iterations 6
minimizer is  [0.99992208 0.99997496 0.99997496 0.99997496 0.99997496] minimum value is 3.5167993666482156e-06 time taken is  15.751688480377197
When n is 5000
total iterations 6
minimizer is  [0.99992143 0.99997476 0.99997476 0.99997476 0.99997476] minimum value is 7.142447349652737e-06 time taken is  110.88473129272461
When n is 7500
total iterations 6
minimizer is  [0.99992017 0.9999713  0.9999713  0.9999713  0.9999713 ] minimum value is 9.962124310022795e-06 time taken is  390.87833166122437
When n is 10000
total iterations 6
minimizer is  [0.9999211  0.99997466 0.99997466 0.99997466 0.99997466] minimum value is 1.4395088425420107e-05 time taken is  829.3313896656036


4. Implement Algorithm 2 for solving minx∈Rn f(x), Use backtracking line search with α0 = 0.9, ρ = 0.5, γ = 0.5.
Take the starting point to be x0 = (0, 0, ..., 0). Take n ∈ {1000, 2500, 5000, 7500, 10000}, find minimizer of the
objective function in each case and compute the time taken by the Newton’s method with backtracking line
search. Tabulate the time taken by Newton’s method for each n.

In [1]:
import numpy as np
from numpy import ma
from math import sqrt, pi
from numpy import exp, cos, sin
from numpy.linalg import norm
def fx(xk):
  length = len(xk)
  sum = 0
  for i in range(length-1):
    sum+= 4*(xk[i]**2 - xk[i+1])**2 + (xk[i]-1)**2
  return sum

def gradient_fx(xk):
  n = len(xk)
  grad = []
  grad.append( 16*xk[0]*(xk[0]**2 - xk[1]) + 2*(xk[0]-1) )
  for i in range(1, n-1):
    grad.append(  -8*(xk[i-1]**2 - xk[i]) + 16*xk[i]*(xk[i]**2 - xk[i+1]) + 2*(xk[i]-1) )
  grad.append(-8*(xk[n-2]**2 - xk[n-1]))
  return np.array(grad)


def hessian_fx(xk):
  n = len(xk)
  hessn = np.zeros((n,n))
  hessn[0][0] = 2 -4*(xk[0] - xk[0]**2) + 2*(1- 2*xk[0])**2
  if n>=2:
    for i in range(1,n):
      hessn[0][0] = hessn[0][0] + 2
      hessn[0][i] = -4*xk[i]
  for i in range(1,n):
    hessn[i][i] = 2 - 4*(xk[0] - xk[i]*2) + 8*(xk[i]**2)
    hessn[i][0] = hessn[0][i]
  return hessn

def dk_fx(xk):
  return np.linalg.inv(hessian_fx(xk))


In [2]:
import time
def get_alpha_newton(xk, alpha0, rho, gamma, Dk):
  alpha = alpha0
  pk = -gradient_fx(xk)
  while fx(xk + alpha*Dk@pk) > (fx(xk) + gamma*alpha*gradient_fx(xk)@Dk@pk):
    alpha = rho*alpha
  return alpha

def newton_method_with_backtracking(x0, tau, alpha0, rho, gamma):
  start_time = time.time()
  xk = np.copy(x0)
  count = 0
  pk = gradient_fx(xk)
  xks = []
  xks.append(xk)
  while norm(pk)>tau:
    Dk = dk_fx(xk)
    alpha = get_alpha_newton(xk, alpha0, rho, gamma, Dk)
    xk = xk - alpha*Dk@pk
    pk = gradient_fx(xk)
    xks.append(xk)
    count += 1
  end_time = time.time()
  time_elapsed = end_time - start_time
  return count, xk, fx(xk), xks, time_elapsed

In [4]:
alpha0 = 0.9
rho = 0.5
gamma = 0.5
ns = [1000,2500,5000, 7500, 10000]
tau = 5
elapsedtimelistforfx_innewton = []
for n in ns:
  x0 = [0 for i in range(n)]
  count, minimizer, minimum, xks, time_elapsed=newton_method_with_backtracking(x0,tau, alpha0, rho, gamma)
  print("for n ",n)
  print("total iterations is",count)
  print( "minimizer is ", minimizer[0:5],  "minimum value is", minimum,  "time taken is", time_elapsed)
  elapsedtimelistforfx_innewton.append(time_elapsed)

KeyboardInterrupt: 

5. Implement Algorithm 2 for solving minx∈Rn g(x), Use backtracking line search with α0 = 0.9, ρ = 0.5, γ = 0.5.
Take the starting point to be x0 = (0, 0, ..., 0). Take n ∈ {1000, 2500, 5000, 7500, 10000}, find minimizer of the
objective function in each case and compute the time taken by the Newton’s method with backtracking line
search. Tabulate the time taken by Newton’s method for each n.

In [5]:
def gx(xk):
  length = len(xk)
  sum = 0
  for i in range(length):
    sum+= (xk[0] - xk[i]**2)**2 + (xk[i]-1)**2
  return sum

def gradient_gx(xk):
  n = len(xk)
  grad = []
  first_grad = 0
  for i in range(1,n):
    first_grad += 2*(xk[0] - xk[i]**2)
  first_grad += 2*(xk[0]-1) + 2*(xk[0] - xk[0]**2)*(1-2*xk[0])
  grad.append( first_grad )
  for i in range(1, n):
    grad.append( -4*(xk[0] - xk[i]**2)*xk[i] + 2*(xk[i]-1) )
  return np.array(grad)

def hessian_gx(xk):
  n = len(xk)
  hessn = np.zeros((n,n))
  hessn[0][0] = 2 -4*(xk[0] - xk[0]**2) + 2*(1- 2*xk[0])**2
  if n>=2:
    for i in range(1,n):
      hessn[0][0] = hessn[0][0] + 2
      hessn[0][i] = -4*xk[i]

  for i in range(1,n):
    hessn[i][i] = 2 - 4*(xk[0] - xk[i]*2) + 8*(xk[i]**2)
    hessn[i][0] = hessn[0][i] #because it is a symmetric matrix
  return hessn


def dk_gx(xk):
  return np.linalg.inv(hessian_gx(xk))

def get_alpha_newton(xk, alpha0, rho, gamma, Dk):
  alpha = alpha0
  pk = -gradient_gx(xk)
  while gx(xk + alpha*Dk@pk) > (gx(xk) + gamma*alpha*gradient_gx(xk)@Dk@pk):
    alpha = rho*alpha
  return alpha

def newton_method_with_backtracking(x0, tau, alpha0, rho, gamma):
  start_time = time.time()
  xk = np.copy(x0)
  count = 0
  pk = gradient_gx(xk)
  xks = []
  xks.append(xk)
  while norm(pk)>tau:
    Dk = dk_gx(xk)
    alpha = get_alpha_newton(xk, alpha0, rho, gamma, Dk)
    xk = xk - alpha*Dk@pk
    pk = gradient_gx(xk)
    xks.append(xk)
    count += 1
  end_time = time.time()
  time_elapsed = end_time - start_time
  return count, xk, gx(xk), xks, time_elapsed


In [6]:
alpha0 = 0.9
rho = 0.5
gamma = 0.5
ns = [1000, 2500, 5000, 7500, 10000]
tau = 1
elapsedtimelistforgx_innewton = []
for n in ns:
  x0 = [0 for i in range(n)]
  count, minimizer, minimum, xks, time_elapsed=newton_method_with_backtracking(x0,tau, alpha0, rho, gamma)
  print(f"for n = {n}")
  print("total iteration is",count)
  print("minimizer is", minimizer[0:5],  "minimum value is", minimum , "time taken is" ,time_elapsed)
  elapsedtimelistforgx_innewton.append(time_elapsed)

for n = 1000
total iteration is 11
minimizer is [0.97208921 0.98594193 0.98594193 0.98594193 0.98594193] minimum value is 0.198946868046022 time taken is 1.4179532527923584
for n = 2500
total iteration is 12
minimizer is [0.98024379 0.99007637 0.99007637 0.99007637 0.99007637] minimum value is 0.24686307604413413 time taken is 22.07803225517273
for n = 5000
total iteration is 13
minimizer is [0.98610682 0.99303202 0.99303202 0.99303202 0.99303202] minimum value is 0.24309609700354048 time taken is 153.50594925880432
for n = 7500
total iteration is 14
minimizer is [0.99025575 0.99511743 0.99511743 0.99511743 0.99511743] minimum value is 0.17896061495935955 time taken is 510.1259968280792
for n = 10000
total iteration is 14
minimizer is [0.9902479  0.99511381 0.99511381 0.99511381 0.99511381] minimum value is 0.23891332569165644 time taken is 1170.4081003665924


6. Compare the time taken by Algorithm 3 - BFGS Method with backtracking line search against the time
taken by Algorithm 2 - Newton’s Method with backtracking line search for each value of n. Plot the time
taken by both methods vs n using different colors. Comment on your observations.

**Observation**

In Algorithm 3 for f(x) for large iteration it take more time than that of g(x).
similarly in Algorithm 3